# Resource Media

In [1]:
import numpy as np
import pandas as pd
import lxml.html
import json
import math

## GetCatalog

In [2]:
file = open('/home/abezpalov/data/rm/category_data.json', 'r')
data = json.load(file)

In [3]:
data

{'Result': 0,
 'ErrorMessage': None,
 'MaterialGroup_Tab': [{'MaterialGroup': 'Z391',
   'MaterialGroupParent': 'Z390',
   'Name': '    2 GB'},
  {'MaterialGroup': 'Z392', 'MaterialGroupParent': 'Z390', 'Name': '    4 GB'},
  {'MaterialGroup': 'Z393', 'MaterialGroupParent': 'Z390', 'Name': '    8 GB'},
  {'MaterialGroup': 'Z999-978',
   'MaterialGroupParent': 'Z383',
   'Name': '    Корпуса SSD/HDD'},
  {'MaterialGroup': 'Z401', 'MaterialGroupParent': 'Z400', 'Name': '   2 GB'},
  {'MaterialGroup': 'Z423', 'MaterialGroupParent': 'Z422', 'Name': '   2 GB'},
  {'MaterialGroup': 'Z999-902',
   'MaterialGroupParent': 'Z422',
   'Name': '   256 GB'},
  {'MaterialGroup': 'Z999-976',
   'MaterialGroupParent': 'Z383',
   'Name': '   32 GB'},
  {'MaterialGroup': 'Z402', 'MaterialGroupParent': 'Z400', 'Name': '   4 GB'},
  {'MaterialGroup': 'Z424', 'MaterialGroupParent': 'Z422', 'Name': '   4 GB'},
  {'MaterialGroup': 'Z403', 'MaterialGroupParent': 'Z400', 'Name': '   8 GB'},
  {'MaterialGroup':

In [4]:
# Преобразуем формат
if data['Result']:
    print(data['ErrorMessage'])
else:
    df = pd.DataFrame(data['MaterialGroup_Tab'])

df = df.rename(columns={'MaterialGroup': 'key',
                        'MaterialGroupParent': 'parent_key',
                        'Name': 'name'})

In [5]:
df

,key,parent_key,name
0,Z391,Z390,2 GB
1,Z392,Z390,4 GB
2,Z393,Z390,8 GB
3,Z999-978,Z383,Корпуса SSD/HDD
4,Z401,Z400,2 GB
...,...,...,...
765,Z999-933,Z023,Широкоформатные аппараты
766,Z606,Z036,Широкоформатные Принтеры
767,Z999-986,Z877,Шпиндели
768,Z589,Z279,Экраны


## GetMaterialData

In [49]:
file = open('/home/abezpalov/data/rm/catalog_data.json', 'r')
data = json.load(file)

In [50]:
data

{'Result': 0,
 'ErrorMessage': None,
 'MaterialData_Tab': {'Item': [{'MaterialID': '0031       ',
    'PartNum': '003R93239',
    'PartNumCust': None,
    'MaterialText': 'Бумага в рулонах 175м XEROX A1+, 620мм, 75г.приклеена к втулке.Грузить кратно 2 рул.',
    'MaterialTextCust': None,
    'MaterialGroup': 'Z990       ',
    'Vendor': 'XEROX',
    'VendorPart': '003R93239',
    'UnitOfMeasurement': 'шт',
    'Multiplicity': 2,
    'Weight': 8.8,
    'Volume': 0.063,
    'Width': 0.3,
    'Length': 0.7,
    'Height': 0.3,
    'Kit': False,
    'VAT': '18%',
    'WebDescription': None,
    'KitParts': [],
    'MaterialCharacteristics': [],
    'BarCodes': [],
    'Certificate': None,
    'Images': None},
   {'MaterialID': '0032       ',
    'PartNum': '003R93240',
    'PartNumCust': None,
    'MaterialText': 'Бумага в рулонах 175м XEROX A0, 841мм, 75г.(аналог 450L90240 в инд.упак.) Грузить кратно 2 рул.',
    'MaterialTextCust': None,
    'MaterialGroup': 'Z990       ',
    'Vendor': '

In [56]:
# Преобразуем формат
if data['Result']:
    print(data['ErrorMessage'])
else:
    df = pd.DataFrame(data['MaterialData_Tab']['Item'])

def get_min_of_quantity(x):
    return 1 if math.isnan(x) else x

def get_step_of_quantity(x):
    return 1 if math.isnan(x) else x

def get_length(row):
    return row['Length'] if row['Length'] >= row['Height'] else row['Height']

def get_width(row):
    return row['Length'] if row['Length'] < row['Height'] else row['Height']

def get_product_metadata(row):
    metadata = dict(row[['Kit', 'KitParts']])
    return json.dumps(metadata)

def get_vat(x):
    return 20 if int(x[:-1]) == 18 else int(x[:-1])

df['min_of_quantity'] = df['Multiplicity'].apply(get_min_of_quantity)
df['step_of_quantity'] = df['Multiplicity'].apply(get_step_of_quantity)
df['length'] = df.apply(get_length, axis=1)
df['width'] = df.apply(get_width, axis=1)
df['metadata'] = df.apply(get_product_metadata, axis=1)
df['vat'] = df['VAT'].apply(get_vat)

df = df.rename(columns={'MaterialID': 'key',
                        'MaterialText': 'name',
                        'MaterialGroup': 'category_key',
                        'Vendor': 'vendor_key',
                        'VendorPart': 'part_number',
                        'UnitOfMeasurement': 'unit_key',
                        'Weight': 'weight',
                        'Height': 'height',
                        'Volume': 'volume',
                       })

df = df.drop(columns=['PartNum', 'PartNumCust', 'MaterialTextCust', 'Multiplicity', 'VAT',
                      'Length', 'Width', 'VAT', 'WebDescription', 'MaterialCharacteristics',
                      'BarCodes', 'Certificate', 'Images', 'Kit', 'KitParts'])

In [58]:
df

,key,name,category_key,vendor_key,part_number,unit_key,weight,volume,height,min_of_quantity,step_of_quantity,length,width,metadata,vat
0,0031,"Бумага в рулонах 175м XEROX A1+, 620мм, 75г.пр...",Z990,XEROX,003R93239,шт,8.800,0.063000,0.300,2.0,2.0,0.700,0.30,"{""Kit"": false, ""KitParts"": []}",20
1,0032,"Бумага в рулонах 175м XEROX A0, 841мм, 75г.(ан...",Z990,XEROX,003R93240,шт,11.600,0.038080,0.952,2.0,2.0,0.952,0.20,"{""Kit"": false, ""KitParts"": []}",20
2,0033,"Бумага в рулонах 175м XEROX A0+, 914мм, 75г.(а...",Z990,XEROX,003R93243,шт,12.700,0.038000,0.200,2.0,2.0,0.950,0.20,"{""Kit"": false, ""KitParts"": []}",20
3,0070,Тонер XEROX DC 535/45/55/WCP 35/45/55 (2 шт) (о),Z222,XEROX,006R01046,шт,3.205,0.020102,0.230,1.0,1.0,0.380,0.23,"{""Kit"": false, ""KitParts"": []}",20
4,0086,Тонер XEROX DC 12/CS 50 голубой (о) 4 шт.,Z223,XEROX,006R90281,шт,2.219,0.003173,31.730,1.0,1.0,31.730,0.01,"{""Kit"": false, ""KitParts"": []}",20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25724,Я600,Комплект беспроводной Defender Skyline 895 Nan...,Z528,DEFENDER,45895,шт,0.610,0.005616,0.060,1.0,1.0,0.480,0.06,"{""Kit"": false, ""KitParts"": []}",20
25725,Я606,Мышь проводная Defender Optimum MS-130 / оптич...,Z531,DEFENDER,52130,шт,0.064,0.000315,0.040,1.0,1.0,0.105,0.04,"{""Kit"": false, ""KitParts"": []}",20
25726,Я607,Мышь проводная Defender Optimum MM-140 / оптич...,Z531,DEFENDER,52140,шт,0.085,0.009000,0.100,1.0,1.0,0.300,0.10,"{""Kit"": false, ""KitParts"": []}",20
25727,Я608,Мышь проводная Defender Optimum MB-150 / оптич...,Z531,DEFENDER,52150,шт,0.045,0.009000,0.100,1.0,1.0,0.300,0.10,"{""Kit"": false, ""KitParts"": []}",20


In [59]:
list(df)

['key',
 'name',
 'category_key',
 'vendor_key',
 'part_number',
 'unit_key',
 'weight',
 'volume',
 'height',
 'min_of_quantity',
 'step_of_quantity',
 'length',
 'width',
 'metadata',
 'vat']

In [42]:
df['BarCodes'].unique()

TypeError: unhashable type: 'list'

In [41]:
df['BarCodes']

0        []
1        []
2        []
3        []
4        []
         ..
25724    []
25725    []
25726    []
25727    []
25728    []
Name: BarCodes, Length: 25729, dtype: object

In [45]:
data = list()

for n, data_row in df.iterrows():
    if data_row['Images']:
        print(data_row['Images'])